In [1]:
import pandas as pd

In [ ]:
# Принято добавлять к названию переменной _df, чтобы было понятно, что это DataFrame
flights_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/flights.csv")
airlines_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/airlines.csv")
airports_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/airports.csv")
planes_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/planes.csv")
weather_df = pd.read_csv("D:/Lern ht code/python_flights_project/data/weather.csv")

flights_df['time_hour'] = pd.to_datetime(flights_df['time_hour'])
weather_df['time_hour'] = pd.to_datetime(weather_df['time_hour'])
weather_df['time_hour'] = weather_df['time_hour'].dt.tz_localize(None)

'''
Преобразуем колонку time_hour в flights_df и weather_df в тип datetime64[ns], потому что был object
Потому что нельзя было делать merge по дате/времени

У flights_df был тип - datetime64[ns]. У weather_df - datetime64[ns, UTC]. Убрал таймзону UTC
'''

'\nПреобразуем колонку time_hour в flights_df и weather_df в тип datetime64[ns], потому что был object\nПотому что нельзя было делать merge по дате/времени\n\nУ flights_df был тип - datetime64[ns]. У weather_df - datetime64[ns, UTC]. Убрал таймзону UTC\n'

In [ ]:
# Сколько было перевозок за 5-й месяц

flights_in_may = airlines_df.merge(flights_df, on='carrier')
flights_in_may = flights_in_may[flights_in_may["month"] == 5]
'''
flights_in_may = flights_in_may[["name", "month"]]  # выбор колонок - Оно типа надо, но на будущее
Но в случае с группировкой .groupby(...).size().reset_index() выборка колонок делается автоматически: 
в результате остаются ключ группировки + агрегат.
'''
result = (
    flights_in_may
    .groupby('name')
    .size()                         # COUNT(*)
    .reset_index(name="count")      # превращаем в колонку
)

result = result.sort_values("count", ascending=False)
print(result)

                           name  count
5      ExpressJet Airlines Inc.    143
12        United Air Lines Inc.    131
8               JetBlue Airways    121
2          Delta Air Lines Inc.    113
1        American Airlines Inc.     91
4                     Envoy Air     65
11              US Airways Inc.     50
3             Endeavor Air Inc.     43
10       Southwest Airlines Co.     27
13               Virgin America     21
0   AirTran Airways Corporation      8
9            Mesa Airlines Inc.      3
6        Frontier Airlines Inc.      1
7        Hawaiian Airlines Inc.      1


In [ ]:
# У какой компании меньше всего опозданий (dep_delay=0)

minimal_delay = airlines_df.merge(flights_df, on='carrier')
minimal_delay = minimal_delay[minimal_delay["dep_delay"] <= 0]

result = (
    minimal_delay
    .groupby('name')['dep_delay']
    .count()                        # COUNT(dep_delay)
    .reset_index(name='not_delay')  # имя колонки как в SQL
    .sort_values('not_delay', ascending=False)
)

print(result)

                           name  not_delay
9               JetBlue Airways        984
3          Delta Air Lines Inc.        957
14        United Air Lines Inc.        940
6      ExpressJet Airlines Inc.        836
2        American Airlines Inc.        627
5                     Envoy Air        496
13              US Airways Inc.        435
4             Endeavor Air Inc.        296
12       Southwest Airlines Co.        172
15               Virgin America         87
0   AirTran Airways Corporation         45
1          Alaska Airlines Inc.         14
7        Frontier Airlines Inc.         13
10           Mesa Airlines Inc.         12
8        Hawaiian Airlines Inc.         11
11        SkyWest Airlines Inc.          1


In [ ]:
all_flights = airlines_df.merge(flights_df, on='carrier')

result = (
    all_flights
    .groupby('name')
    .size()
    .sort_values(ascending=False)
)

print(result)

'''
--Подсчёт сколько всего было полётов всех
SELECT al.name, COUNT(*)
FROM airlines AS al
INNER JOIN flights as fl
USING (carrier)
GROUP BY al.name
ORDER BY COUNT(*) DESC
/* Мы получили, что самая не опаздывающая компания имеет 984 не опозданий. 
Всего у неё 1636 рейс. 60.15% рейсов не опаздывают */;

'''

name
United Air Lines Inc.          1741
ExpressJet Airlines Inc.       1643
JetBlue Airways                1636
Delta Air Lines Inc.           1425
American Airlines Inc.          958
Envoy Air                       769
US Airways Inc.                 592
Endeavor Air Inc.               521
Southwest Airlines Co.          386
Virgin America                  150
AirTran Airways Corporation      97
Alaska Airlines Inc.             23
Frontier Airlines Inc.           23
Mesa Airlines Inc.               20
Hawaiian Airlines Inc.           15
SkyWest Airlines Inc.             1
dtype: int64


'\n--Подсчёт сколько всего было полётов всех\nSELECT al.name, COUNT(*)\nFROM airlines AS al\nINNER JOIN flights as fl\nUSING (carrier)\nGROUP BY al.name\nORDER BY COUNT(*) DESC\n/* Мы получили, что самая не опаздывающая компания имеет 984 не опозданий. \nВсего у неё 1636 рейс. 60.15% рейсов не опаздывают */;\n\n'

In [ ]:
weather_impact = flights_df.merge(weather_df, on=['time_hour', 'origin'])
weather_impact = weather_impact[weather_impact['dep_delay']>0]


result = (
    weather_impact
    .groupby('visib')
    .agg(
        avg_delay=('dep_delay', 'mean'),      # AVG(dep_delay)
        number_of_flights=('dep_delay', 'count')  # COUNT(dep_delay)
    )
    .reset_index() 
    .sort_values('visib', ascending=False)
)

print(result)


'''
-- Как погода влияла на опоздание. Чем меньше visib, тем хуже видно
SELECT we.visib, AVG(fl.dep_delay) AS avg_delay, COUNT(*) AS number_of_flights
FROM flights AS fl
INNER JOIN weather AS we 
ON fl.time_hour = we.time_hour AND fl.origin = we.origin
WHERE fl.dep_delay > 0
GROUP BY we.visib
ORDER BY we.visib DESC
/* На удивление, корреляция арактически незаметна*/;
'''

    visib   avg_delay  number_of_flights
19  10.00   36.826693               3012
18   9.00   46.164474                152
17   8.00   56.980198                101
16   7.00   39.905660                106
15   6.00   50.333333                 75
14   5.00   62.370370                 81
13   4.00   58.121951                 41
12   3.00   48.816667                 60
11   2.50   63.205128                 39
10   2.00   38.913043                 46
9    1.75  113.500000                  4
8    1.50   69.971429                 35
7    1.25   66.714286                  7
6    1.00   56.947368                 19
5    0.75   47.750000                  4
4    0.50   58.611111                 18
3    0.25   28.000000                 18
2    0.12   98.666667                 12
1    0.06   68.000000                  1
0    0.00   52.800000                  5


'\n-- Как погода влияла на опоздание. Чем меньше visib, тем хуже видно\nSELECT we.visib, AVG(fl.dep_delay) AS avg_delay, COUNT(*) AS number_of_flights\nFROM flights AS fl\nINNER JOIN weather AS we \nON fl.time_hour = we.time_hour AND fl.origin = we.origin\nWHERE fl.dep_delay > 0\nGROUP BY we.visib\nORDER BY we.visib DESC\n/* На удивление, корреляция арактически незаметна*/;\n'

In [ ]:
flights_with_airlines = flights_df.merge(airlines_df, on='carrier')
flights_per_company = flights_with_airlines.merge(planes_df, on='tailnum')
result = (
    flights_per_company
    .groupby('name')['year_y']  # year_y так как это год выпуска самолёта.
    .mean()                     # в flights_df тоже есть колонка year, она отвечает за год рейса
    .round(0)                   # округляет year_y
    .reset_index()
    .sort_values('year_y', ascending=False)
)

print(result)

'''
--Средний возраст у самолётов каждой авиакомпании и кол-во рейсов
SELECT al.name AS airline_name, ROUND(AVG(p.year), 0) AS avg_plane_age
FROM flights AS fl
INNER JOIN airlines AS al 
ON fl.carrier = al.carrier
INNER JOIN planes AS p 
ON fl.tailnum = p.tailnum
GROUP BY al.name
ORDER BY avg_plane_age DESC;
'''

                           name  year_y
8        Hawaiian Airlines Inc.  2012.0
11        SkyWest Airlines Inc.  2010.0
1          Alaska Airlines Inc.  2009.0
15               Virgin America  2009.0
7        Frontier Airlines Inc.  2008.0
9               JetBlue Airways  2007.0
4             Endeavor Air Inc.  2006.0
10           Mesa Airlines Inc.  2004.0
13              US Airways Inc.  2004.0
12       Southwest Airlines Co.  2004.0
6      ExpressJet Airlines Inc.  2002.0
0   AirTran Airways Corporation  2001.0
14        United Air Lines Inc.  2000.0
3          Delta Air Lines Inc.  1997.0
2        American Airlines Inc.  1987.0
5                     Envoy Air  1978.0


'\n--Средний возраст у самолётов каждой авиакомпании и кол-во рейсов\nSELECT al.name AS airline_name, ROUND(AVG(p.year), 0) AS avg_plane_age\nFROM flights AS fl\nINNER JOIN airlines AS al \nON fl.carrier = al.carrier\nINNER JOIN planes AS p \nON fl.tailnum = p.tailnum\nGROUP BY al.name\nORDER BY avg_plane_age DESC;\n'